In [ ]:
import requests
import os
from pprint import pprint
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from requests.auth import HTTPBasicAuth
import json
from nameparser import HumanName

In [ ]:
url = "https://xxxx"
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

#f = requests.get(url, headers = headers)

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
allfields = json.loads(response.text)

In [ ]:
df = pd.DataFrame(allfields)

In [ ]:
df.head()

In [ ]:
import re

In [ ]:
%%script false

#this function creates separate columns for each contact and does not work with explode.
#given a list of collaborators, produce the following:
#email1, email2, ... email-all. These are all email links.
emailListOfDicts = []

def extractEmails1(emailList):
    emailDict = {}
    i=1
    allemails = []
    allnames = []
    for email in emailList:
        matches = re.findall(r'(.*)<a.*>(.*)</a>', email)
        if matches:
            (name, email) = matches[0]
            #insert each 
            emailDict['Contact Name'+str(i)] = name.strip()
            emailDict['Contact Email'+str(i)] = email.strip()
            emailDict['Contact Link'+str(i)] = '=HYPERLINK("mailto:{}?subject=Hi from XXXXXXX &body=Hi {}","{}")'.format(email.strip(), name.strip(), name.strip())
            
            allemails.append(email.strip())
            allnames.append(name.strip())
            i = i + 1
    if i > 1:
        emailDict['All Contacts Email'] = ','.join(allemails)
        emailDict['All Contacts Link'] = '=HYPERLINK("mailto:{}?subject=Hi from XXXXXXX&body=Hi {}", "{}")'.format(','.join(allemails), ', '.join(allnames), ', '.join(allnames))
    
    #this last step must be done even if we did not add any new columns!
    emailListOfDicts.append(emailDict)
        

In [ ]:
#given a list of collaborators, produce the following:
#email1, email2, ... email-all. These are all email links.
emailListOfDicts = []

def extractEmails(emailList):

    allemails = []
    allnames = []
    alllinks = []
    allfirst = []
    allmiddle = []
    alllast = []
    alltitle = []
    allsuffix = []
    
    for email in emailList:
        matches = re.findall(r'(.*)<a.*>(.*)</a>', email)
        if matches:
            (name, email) = matches[0]
            if name:
                name_parts = HumanName(name)
                alltitle.append(name_parts.title)
                allfirst.append(name_parts.first)
                allmiddle.append(name_parts.middle)
                alllast.append(name_parts.last)
                allsuffix.append(name_parts.suffix)
            else:
                alltitle.append("")
                allfirst.append("")
                allmiddle.append("")
                alllast.append("")
                allsuffix.append("")

            allemails.append(email.strip())
            allnames.append(name.strip())
            alllinks.append('=HYPERLINK("mailto:{}?subject=Hi from XXXXXX&body=Hi {}","{}")'.format(email.strip(), name.strip(), name.strip()))

    emailDict = {'Contact Full Name':allnames, 'Contact Title':alltitle, 
                 'Contact First Name':allfirst, 'Contact Middle Name':allmiddle,
                 'Contact Last Name':alllast, 'Contact Suffix':allsuffix,
                 'Contact Email':allemails, 'Contact Link':alllinks}
    emailListOfDicts.append(emailDict)

In [ ]:
df['collaborators'].apply(extractEmails)

In [ ]:
#take a dictionary and add columns
df2 = pd.DataFrame(emailListOfDicts)

In [ ]:
df3 = pd.concat([df, df2], axis=1)

In [ ]:
df3 = df3.drop(columns=['lead', 'country_code', 'continent', 'state_name', 'id'])

In [ ]:
df4 = df3.explode(['Contact Full Name', 'Contact Title', 
                 'Contact First Name', 'Contact Middle Name',
                 'Contact Last Name', 'Contact Suffix', 'Contact Email', 'Contact Link'])

In [ ]:
writer = pd.ExcelWriter('leads.xlsx', mode = 'w')
df4.to_excel(writer, sheet_name='list', index=False, freeze_panes=(1,1))

#format this
workbook = writer.book
worksheet = writer.sheets['list']

header_format = workbook.add_format({'text_wrap': True})

(max_row, max_col) = df4.shape
worksheet.set_column(0, max_col, 25, header_format)


writer.save()